# Simulations to test GroupHug vs. 'one person, one vote' in case of maximum disagreement between voter groups.

The perform_comparison function below configures a GroupHug mechanism with a specified number of four different voter types - experts, intellectuals, participants and uneducated community members. In this simulation, each group is in favor of a different candidate. The experts prefer the expert candidate, the intellectuals prefer the intellectual candidate, ...and so on.

Furthermore, a SingleChoiceWeightedPlurality mechanism is set up with the same total number of voters - and uniform weights.

Each mechanism calculates a set of scores and announces a winner.

In [1]:
import sys
sys.path.append("..")

from mechanisms.group_hug_mechanism import GroupHug
from mechanisms.single_choice_weighted_plurality import SingleChoiceWeightedPlurality

def str_dict(d):
    return str(dict(sorted(d.items())))

def perform_comparison(gh, ecount, icount, pcount, ucount):

    expert_type = {"FELLOWSHIP_COMM": True}
    intellectual_type = {"FUND_MOD_1": True}
    participant_type = {"LIVE_TRACK_1": True}
    uneducated_type = {}

    voters = {}
    voters_choices = {}

    for i in range(ecount):
        voters["e" + str(i)] = expert_type
        voters_choices["e" + str(i)] = "Expert candidate"

    for i in range(icount):
        voters["i" + str(i)] = intellectual_type
        voters_choices["i" + str(i)] = "Intellectual candidate"

    for i in range(pcount):
        voters["p" + str(i)] = participant_type
        voters_choices["p" + str(i)] = "Participant candidate"
        
    for i in range(ucount):
        voters["u" + str(i)] = uneducated_type
        voters_choices["u" + str(i)] = "Uneducated candidate"

    uniform_voters = {id: {"weight": 1.0} for id in voters.keys()}

    print("VOTERS: " + str(ecount) + " experts, " + str(icount) + " intellectuals, " + str(pcount) + " participants, " + str(ucount) + " uneducated.")

    w_gh, r_gh = gh.calculate(voters, voters_choices)

    print("\nGroupHug Winner: " + str(w_gh))
    print("GroupHug Scores: " + str_dict(r_gh) + "\n")

    SCWP = SingleChoiceWeightedPlurality()
    w_scwp, r_scwp = SCWP.calculate(uniform_voters, voters_choices)

    print("Popularity contest winner: " + str(w_scwp))
    print("Popularity contest scores: " + str_dict(r_scwp))

Let's first use a default GroupHug mechanism (all voter groups have equal weight, default NFT weights are used.)

In [2]:
GH_def = GroupHug()

In our first example, below, only experts and uneducated community members participate (no intellectuals or participants.)

We see that a small expert group override a very large group of uneducated voters because the default GroupHug caps the influence of any of the N participating voter groups at 1/N, and the experts also influence the community vote:

In [3]:
perform_comparison(GH_def, 1, 0, 0, 500)

VOTERS: 1 experts, 0 intellectuals, 0 participants, 500 uneducated.

Experts: {'Expert candidate': 100.0, 'Uneducated candidate': 0.0}
Intellectuals: {'Expert candidate': 0, 'Uneducated candidate': 0}
Participants: {'Expert candidate': 0, 'Uneducated candidate': 0}
Community: {'Expert candidate': 0.2, 'Uneducated candidate': 99.8}

GroupHug Winner: Expert candidate
GroupHug Scores: {'Expert candidate': 50.1, 'Uneducated candidate': 49.9}

Popularity contest winner: Uneducated candidate
Popularity contest scores: {'Expert candidate': 1.0, 'Uneducated candidate': 500.0}


If, at this point, a single 'intellectual' enters a vote (disagreeing with both experts and uneducated community members), the expert and the intellectual have equal weight. 

The tie will be resolved by the experts, according to the design.

In [4]:
perform_comparison(GH_def, 1, 1, 0, 100)

VOTERS: 1 experts, 1 intellectuals, 0 participants, 100 uneducated.

Experts: {'Expert candidate': 100.0, 'Intellectual candidate': 0.0, 'Uneducated candidate': 0.0}
Intellectuals: {'Expert candidate': 0.0, 'Intellectual candidate': 100.0, 'Uneducated candidate': 0.0}
Participants: {'Expert candidate': 0, 'Intellectual candidate': 0, 'Uneducated candidate': 0}
Community: {'Expert candidate': 1.0, 'Intellectual candidate': 1.0, 'Uneducated candidate': 98.0}
Tie. We'll ask the experts to resolve it.

GroupHug Winner: Expert candidate
GroupHug Scores: {'Expert candidate': 33.7, 'Intellectual candidate': 33.7, 'Uneducated candidate': 32.7}

Popularity contest winner: Uneducated candidate
Popularity contest scores: {'Expert candidate': 1.0, 'Intellectual candidate': 1.0, 'Uneducated candidate': 100.0}


Note that in the default GroupHug mechanism, 1 expert has the same power as 1 student. 

But we can change this by using custom group weights...

In [5]:
GH_custom_a = GroupHug(experts_group_weight = 30,
                     intellectuals_group_weight = 25,
                     participants_group_weight = 25,
                     community_group_weight = 20)


With the custom GroupHug mechanism defined above, 1 expert outweights 1 intellectual:

In [6]:
perform_comparison(GH_custom_a, 1, 1, 0, 100)

VOTERS: 1 experts, 1 intellectuals, 0 participants, 100 uneducated.

Experts: {'Expert candidate': 100.0, 'Intellectual candidate': 0.0, 'Uneducated candidate': 0.0}
Intellectuals: {'Expert candidate': 0.0, 'Intellectual candidate': 100.0, 'Uneducated candidate': 0.0}
Participants: {'Expert candidate': 0, 'Intellectual candidate': 0, 'Uneducated candidate': 0}
Community: {'Expert candidate': 1.0, 'Intellectual candidate': 1.0, 'Uneducated candidate': 98.0}

GroupHug Winner: Expert candidate
GroupHug Scores: {'Expert candidate': 40.3, 'Intellectual candidate': 33.6, 'Uneducated candidate': 26.1}

Popularity contest winner: Uneducated candidate
Popularity contest scores: {'Expert candidate': 1.0, 'Intellectual candidate': 1.0, 'Uneducated candidate': 100.0}


In fact, 3 experts outweight 30 students and 100 uneducated:

In [7]:
perform_comparison(GH_custom_a, 3, 30, 0, 100)

VOTERS: 3 experts, 30 intellectuals, 0 participants, 100 uneducated.

Experts: {'Expert candidate': 100.0, 'Intellectual candidate': 0.0, 'Uneducated candidate': 0.0}
Intellectuals: {'Expert candidate': 0.0, 'Intellectual candidate': 100.0, 'Uneducated candidate': 0.0}
Participants: {'Expert candidate': 0, 'Intellectual candidate': 0, 'Uneducated candidate': 0}
Community: {'Expert candidate': 2.3, 'Intellectual candidate': 22.6, 'Uneducated candidate': 75.2}

GroupHug Winner: Expert candidate
GroupHug Scores: {'Expert candidate': 40.6, 'Intellectual candidate': 39.3, 'Uneducated candidate': 20.0}

Popularity contest winner: Uneducated candidate
Popularity contest scores: {'Expert candidate': 3.0, 'Intellectual candidate': 30.0, 'Uneducated candidate': 100.0}


But there is still a limit to the experts' power. 

For example, 50 students (intellectuals) outweigh 3 experts:

In [8]:
perform_comparison(GH_custom_a, 3, 50, 0, 100)

VOTERS: 3 experts, 50 intellectuals, 0 participants, 100 uneducated.

Experts: {'Expert candidate': 100.0, 'Intellectual candidate': 0.0, 'Uneducated candidate': 0.0}
Intellectuals: {'Expert candidate': 0.0, 'Intellectual candidate': 100.0, 'Uneducated candidate': 0.0}
Participants: {'Expert candidate': 0, 'Intellectual candidate': 0, 'Uneducated candidate': 0}
Community: {'Expert candidate': 2.0, 'Intellectual candidate': 32.7, 'Uneducated candidate': 65.4}

GroupHug Winner: Intellectual candidate
GroupHug Scores: {'Expert candidate': 40.5, 'Intellectual candidate': 42.0, 'Uneducated candidate': 17.4}

Popularity contest winner: Uneducated candidate
Popularity contest scores: {'Expert candidate': 3.0, 'Intellectual candidate': 50.0, 'Uneducated candidate': 100.0}


Of course, we can move the threshold by changing the group weights slightly more in favor of the experts...

In [9]:
GH_custom_b = GroupHug(experts_group_weight = 32,
                     intellectuals_group_weight = 24,
                     participants_group_weight = 24,
                     community_group_weight = 20)

perform_comparison(GH_custom_b, 3, 50, 0, 100)

VOTERS: 3 experts, 50 intellectuals, 0 participants, 100 uneducated.

Experts: {'Expert candidate': 100.0, 'Intellectual candidate': 0.0, 'Uneducated candidate': 0.0}
Intellectuals: {'Expert candidate': 0.0, 'Intellectual candidate': 100.0, 'Uneducated candidate': 0.0}
Participants: {'Expert candidate': 0, 'Intellectual candidate': 0, 'Uneducated candidate': 0}
Community: {'Expert candidate': 2.0, 'Intellectual candidate': 32.7, 'Uneducated candidate': 65.4}

GroupHug Winner: Expert candidate
GroupHug Scores: {'Expert candidate': 42.6, 'Intellectual candidate': 40.2, 'Uneducated candidate': 17.2}

Popularity contest winner: Uneducated candidate
Popularity contest scores: {'Expert candidate': 3.0, 'Intellectual candidate': 50.0, 'Uneducated candidate': 100.0}


Now, it takes even more student votes to override the experts' opinion:

In [10]:
perform_comparison(GH_custom_b, 3, 75, 0, 100)

VOTERS: 3 experts, 75 intellectuals, 0 participants, 100 uneducated.

Experts: {'Expert candidate': 100.0, 'Intellectual candidate': 0.0, 'Uneducated candidate': 0.0}
Intellectuals: {'Expert candidate': 0.0, 'Intellectual candidate': 100.0, 'Uneducated candidate': 0.0}
Participants: {'Expert candidate': 0, 'Intellectual candidate': 0, 'Uneducated candidate': 0}
Community: {'Expert candidate': 1.7, 'Intellectual candidate': 42.1, 'Uneducated candidate': 56.2}

GroupHug Winner: Intellectual candidate
GroupHug Scores: {'Expert candidate': 42.6, 'Intellectual candidate': 42.7, 'Uneducated candidate': 14.8}

Popularity contest winner: Uneducated candidate
Popularity contest scores: {'Expert candidate': 3.0, 'Intellectual candidate': 75.0, 'Uneducated candidate': 100.0}
